In [ ]:
#this script will scrape through google image search using selenium
#also creates a json file by create_json()

In [61]:
#参考
#https://stackoverflow.com/questions/20716842/python-download-images-from-google-image-search
import json
import os
from hashlib import md5

import urllib.parse
from urllib.request import Request, urlopen

from selenium import webdriver

In [76]:
def create_json(filename,imgurl,siteurl,imgtype,sitetitle,imgtitle,changeflag):
    d = dict()
    text = '' + imgurl
    m = md5()
    m.update(text.encode('utf-8'))
    d['hash'] = m.hexdigest()
    d['filename'] = filename
    d['sitetitle'] = sitetitle
    d['imgtitle'] = imgtitle
    d['imgurl'] = imgurl
    d['siteurl'] = siteurl
    d['imgtype'] = imgtype
    d['flag'] = changeflag
    json_output = json.dumps(d,ensure_ascii=False)
    return json_output

searchterm = 'tomato Alternaria solani' # will also be the name of the folder

searchdir = searchterm.replace(" ","_")
searchdir = searchterm.replace("\"","")
url = "https://www.google.com/search?gl=us&hl=en&pws=0&gws_rd=cr&q="+searchterm+"&source=lnms&tbm=isch"
browser = webdriver.Firefox()
#browser = webdriver.PhantomJS()
browser.get(url)

counter = 0
succounter = 0

if not os.path.exists(searchdir):
    os.mkdir(searchdir)

button_xpath = '//input[@id="smb"]'

for _ in range(500):
    browser.execute_script("window.scrollBy(0,10000)")
    try:
        browser.find_element_by_xpath(button_xpath).click()
        print ("clicked show more results button")
    except:
        pass

for x in browser.find_elements_by_xpath('//div[contains(@class,"rg_meta")]'):
    counter += 1
    changeflag = 0
    print ("Total Count:", counter)

    imgurl = json.loads(x.get_attribute('innerHTML'))["ou"]
    siteurl = json.loads(x.get_attribute('innerHTML'))["ru"]
    imgtype = json.loads(x.get_attribute('innerHTML'))["ity"]
    sitetitle = json.loads(x.get_attribute('innerHTML'))["pt"]
    imgtitle = json.loads(x.get_attribute('innerHTML'))["s"]
    print ("imageURL:",imgurl)
    print ("siteURL:",siteurl)
    print ("imgtype:",imgtype)
    print ("sitetitle:",sitetitle)
    print ("imgtitle:",imgtitle)

    try:
        #save image to file
        with urllib.request.urlopen(imgurl) as response:
            raw_img = response.read()
            filename = searchdir + "_" + str(succounter) + "." + imgtype
            File = open(os.path.join(searchdir,filename), "wb")
            #print (os.path.join(searchdir , filename))
            File.write(raw_img)
            File.close()

        succounter += 1
        
        #create and write json object to file
        if imgtype is not None:
            if imgtype not in ["jpg","jpeg","tif","tiff","gif"]:
                imgtype = "jpg"
                changeflag = 1
            data = create_json(filename,imgurl,siteurl,
                               imgtype,sitetitle,imgtitle,changeflag)
            with open(os.path.join(searchdir,searchdir+'.json'), 'a') as f:
                if succounter > 1:
                    f.write(",\n")
                f.write(data)

        print ("Succsessful Count:", succounter)
        print ("__________________\n")
    
    except Exception as e:
        print ("could not retreive the image, and was not recorded to json because of the following reason.")
        print (e)
        print ("__________________\n")
    #stopping loop for demo usage
    if succounter == 10:
        break
browser.quit()
print ("done")

clicked show more results button
Total Count: 1
imageURL: http://c8.alamy.com/comp/ACG8W6/early-blight-or-targfet-spot-alternaria-solani-lesions-on-a-tomato-ACG8W6.jpg
siteURL: http://www.alamy.com/stock-photo-early-blight-or-target-spot-alternaria-solani-lesions-on-a-tomato-27744668.html
imgtype: jpg
sitetitle: Early blight or target spot (Alternaria solani) lesions on a ...
imgtitle: ... Early blight or targfet spot Alternaria solani lesions on a tomato  leaflet - Stock Photo
Succsessful Count: 1
__________________

Total Count: 2
imageURL: https://upload.wikimedia.org/wikipedia/commons/thumb/8/8e/Alternaria_solani_-_stem_lesions.jpg/220px-Alternaria_solani_-_stem_lesions.jpg
siteURL: https://en.wikipedia.org/wiki/Alternaria_solani
imgtype: jpg
sitetitle: Alternaria solani - Wikipedia
imgtitle: Stem lesion of Alternaria solani
Succsessful Count: 2
__________________

Total Count: 3
imageURL: https://www.apsnet.org/publications/imageresources/PublishingImages/2013/fi00187.jpg
siteURL: